**26th June 2021 to 1st July 2021**

# Problems covered

1. Check if Regular Expression matches.
2. Move zeros while maintaining order.
3. Check which words are present in the grid.
4. TwoSum; given array of numbers, and a target, return indices of two numbers which add up to the target.
5. Generate all unique non-empty letter combinations for a string.
6. Given a number of parenthesis pairs, find all valid arrangements.

## 1. Check if Regular Expression matches.

You only have two *symbols*:
* `.` matches to any char
* `x*` matches to 0 or more times for `x`
* `.*` matches to 0 or more times for any char

Given a string `aabc` and a pattern `a.*`, verify if the string matches the *entire* pattern.

In [27]:
from string import ascii_lowercase
from functools import lru_cache


class RegexpMatches:
    def solve(self, string, p):
        if len(string) == 0:
            return len(p) == 0 or (len(p)==2 and p[-1]=='*')
        
        if len(p) == 0:
            return False
        
        letters = set(ascii_lowercase)
        
        pattern = []
        # Reverse over `p` to identify pattern blocks.
        i = len(p)-1
        while i >= 0:
            if p[i] in letters or p[i] == '.':
                pattern.append(p[i])
                i -= 1
            else:
                pattern.append(p[i-1:i+1])
                i -= 2
                
        pattern = pattern[::-1]
        if len(string) < len([x for x in pattern if x in letters or x == '.']):
            return False
        
        # print(pattern)
        
        @lru_cache(maxsize=None)
        def match(i, j):
            """
            i: String
            j: Pattern
            """
            if i == len(string) and j == len(pattern):
                return True
            else:
                if i < len(string) and j < len(pattern):
                    if pattern[j] == '.':
                        return match(i+1, j+1)
                    elif pattern[j] in letters:
                        return pattern[j] == string[i] and match(i+1, j+1)
                    else:
                        # You have x* or .*
                        char = pattern[j][0]
                        # Try skipping.
                        skipped = match(i, j+1)
                        if skipped:
                            return True
                        else:
                            if char == '.':
                                return match(i+1, j)
                            else:
                                return char == string[i] and match(i+1, j)
                            
                elif i < len(string):
                    # You are out of patterns.
                    return False  # You cannot add anything to the string.
                
                else:
                    # You are out of string.
                    if pattern[j] in letters or pattern[j] == '.':
                        return False
                    else:
                        return match(i, j+1)
                    
        return match(0, 0)

In [28]:
o = RegexpMatches()
o.solve('mississippi', 'mis*i.*p*.')

True

## 2. Move zeros while maintaining order.

Input : `[1, 5, -1, 7, 0, 8, 2, 0, 0]`

Output: `[1, 5, -1, 7, 8, 2, 0, 0, 0]`

Do it in-place and do not use extra memory!

In [29]:
class MoveZeros:
    def solve(self, array):
        if len(array) <= 1:
            return array
        
        non = 0
        z = 0
        
        while non < len(array):
            if z == non:
                non += 1
                
            if array[non] == 0:
                non += 1
                
            if array[z] != 0:
                z += 1
            
            if z < non and z < len(array) and non < len(array) and array[z] == 0 and array[non] != 0:
                array[z], array[non] = array[non], array[z]

In [31]:
array = [1, 5, -1, 7, 0, 8, 2, 0, 0]
o = MoveZeros()
o.solve(array)
array

[1, 5, -1, 7, 8, 2, 0, 0, 0]

## 3. Check which words are present in the grid.

You are given a grid of characters, and a list of words.

Return which words can be constructed from the grid.

You are permitted to move horizontally and vertically (single block only), and each block can only be used at most *once* for a word.

**OPTIMIZATION**

Most important thing is that with the TRIE, you don't have to search for each word from scratch.

Also, because you keep deleting nodes once you find the leaf (i.e. `#`), over time the search space in the TRIE would decrease.

But the time complexity: $O(B \times 3^L)$ is an upper-bound: B ==> number of cells in grid; L ==> Max word length.

In [45]:
class CheckWordsGrid:
    def solve(self, grid, words):
        """
        H E L L W
        E L L O B
        S E P V C
        
        {HELLO, ELLO, SELL, VOW}
        """
        
        rows = len(grid)
        if rows == 0:
            return []
        cols = len(grid[0])
        if cols == 0:
            return []
        
        if len(words) == 0:
            return []
        
        ans = []
        
        trie = {}
        for w in words:
            node = trie
            for char in w:
                if char in node:
                    node = node[char]
                else:
                    node[char] = {}
                    node = node[char]
                    
            node['#'] = w
            
        # print(trie)
        
        def search(i, j, node):
            current = grid[i][j]
            # print(current, node)
            if '#' in node[current]:
                ans.append(node[current]['#'])
                del node[current]['#']
                
            # This already exists in the current node.
            # Mark this location as used.
            grid[i][j] = None
            
            for x, y in [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]:
                if x >= 0 and x < rows and y >= 0 and y < cols and\
                grid[x][y] is not None and grid[x][y] in node[current]:
                    search(x, y, node[current])
                    
            grid[i][j] = current
            if not node[current]:
                del node[current]
                    
        
        for i in range(rows):
            for j in range(cols):
                char = grid[i][j]
                if char in trie:
                    search(i, j, trie)
                    
        return ans

In [46]:
"""
        H E L L W
        E L L O B
        S E P V C
        
        {HELLO, ELLO, SELL, VOW}
"""

o = CheckWordsGrid()
o.solve(
    [
        list('HELLW'),
        list('ELLOB'),
        list('SEPVC')
    ],
    ['HELLO', 'ELLO', 'ELE', 'SELL', 'VOW']
)

['HELLO', 'ELE', 'ELLO', 'SELL']

## 4. TwoSum; given array of numbers, and a target, return indices of two numbers which add up to the target.

```
input: [2, 10, 7, 15]
target: 9
```

How?
* Can you assume list is sorted?
    * If yes, then you can use two pointers (first, last)
    * If no, then sort the list, while tracking indices, and then run two-pointers



In [54]:
class TwoSum:
    def solve(self, array, target):
        positions = [(array[i], i) for i in range(len(array))]
        positions.sort()
        
        i = 0
        j = len(positions)-1
        while i < j:
            a = positions[i][0]
            b = positions[j][0]
            total = a + b
            
            if total == target:
                return positions[i][1], positions[j][1]
            elif total > target:
                j -= 1
            else:
                i += 1
                
        return None
    
    def linear(self, array, target):
        complement = {}
        for ix, n in enumerate(array):
            diff = target - n
            if diff in complement:
                return ix, complement[diff]
            else:
                complement[n] = ix
                
        return None

In [56]:
o = TwoSum()
assert o.solve([2, 10, 7, 15], 9) == (0, 2) or o.solve([2, 10, 7, 15], 9) == (2, 0)
assert o.linear([2, 10, 7, 15], 9) == (0, 2) or o.linear([2, 10, 7, 15], 9) == (2, 0)

## 5. Generate all unique non-empty letter combinations for a string.

```
input: "AABC"
output: [
A, B, C,
AA, AB, AC, BC,
AAB, AAC, ABC
]
```

**Time Complexity**: $\sum_{k \in [1, n]} k \times nCk$

The summation without the $k$ multiplier will be $2^n$. So the time complexity is more than that.

We also know that the recursive relation is $T(n) = T(n-1) \times (n-1)$, which is less than $n^n$.


**Space Complexity**: Set for holding {$nCk \times k$} elements, where $k \in [1, n]$

In [63]:
class GenerateUnique:
    def solve(self, string):
        """
        AABC
        
        Start from each position: (0, 1, 2, 3)
            Run for loop that goes from start, to end.
                THIS WILL NOT WORK => Only selects contiguous substrings; AC will never be formed.
                
        Start from 0
            Make a temp string by adding next character to this index.
                Then treat this as the base, and recursively call for the next position.
                
        0: "A" => A+A A+B A+C
                   |________ AA => AA+B, AA+C
                                    |________ AAB => AAB + C
                
        """
        unique = set()
        
        def generate(start, base):
            for i in range(start, len(string)):
                temp = base+string[i]
                unique.add(temp)
                generate(i+1, temp)
                
        generate(0, '')
        
        return unique

In [64]:
o = GenerateUnique()
o.solve('AABC')

{'A', 'AA', 'AAB', 'AABC', 'AAC', 'AB', 'ABC', 'AC', 'B', 'BC', 'C'}

In [72]:
class ValidParenthesis:
    def solve(self, n):
        self.ans = 0
        
        def check(temp):
            close = 0
            for char in temp:
                if close > 0:
                    return False
                
                if char == ')':
                    close += 1
                else:
                    close -= 1
                    
            return close == 0
        
        def generate(temp):
            if len(temp) == 2*n:
                if check(temp):
                    self.ans += 1
                    
            else:
                temp.append('(')
                generate(temp)
                temp.pop()
                
                temp.append(')')
                generate(temp)
                temp.pop()
                
        generate([])
        return self.ans

In [73]:
o = ValidParenthesis()
o.solve(3)

5